In [ ]:
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
import os
from dotenv import load_dotenv
import uvicorn
from fastapi import FastAPI, Request, HTTPException
# Load environment variables
load_dotenv()

: 

In [ ]:
# Models
class Blog(BaseModel):
    """Represents the state of a blog post."""
    title: str = Field(default="", description="The title of the blog post")
    content: str = Field(default="", description="The content of the blog post")

class BlogState(BaseModel):
    """Represents the state of the blog."""
    topic: str
    blog: Blog = Blog()
    current_language: str = "english"

In [ ]:
# LLM Setup
class GroqLLM:
    def __init__(self):
        api_key = os.getenv("GROQ_API_KEY")
        if not api_key:
            raise ValueError("GROQ_API_KEY environment variable not set")
        
        self.llm = ChatGroq(
            api_key=api_key,
            model="llama3-8b-8192",
            max_retries=3,
            timeout=60,
        )
    
    def get_llm(self):
        """Returns the Groq client instance."""
        return self.llm


In [ ]:
# Nodes
class BlogNode:
    def __init__(self, llm):
        self.llm = llm

    def title_creation(self, state: BlogState) -> dict:
        """Create the title for the blog."""
        if state.topic:
            prompt = """You are an expert blog content writer. Use Markdown formatting. 
                      Generate a blog title for the {topic}. This title should be creative and SEO friendly"""
            system_message = prompt.format(topic=state.topic)
            response = self.llm.invoke(system_message)
            return {"blog": {"title": response.content}}
        return {}

    def content_generation(self, state: BlogState) -> dict:
        """Generate blog content."""
        if state.topic:
            system_prompt = """You are expert blog writer. Use Markdown formatting.
                            Generate a detailed blog content with detailed breakdown for the {topic}"""
            system_message = system_prompt.format(topic=state.topic)
            response = self.llm.invoke(system_message)
            return {"blog": {"title": state.blog.title, "content": response.content}}
        return {}

    def translation(self, state: BlogState) -> dict:
        """Translate the content to the specified language."""
        translation_prompt = """Translate the following content into {current_language}.
                            - Maintain the original tone, style, and formatting.
                            - Adapt cultural references and idioms to be appropriate for {current_language}.

                            ORIGINAL CONTENT:
                            {blog_content}"""
        
        messages = [
            HumanMessage(translation_prompt.format(
                current_language=state.current_language,
                blog_content=state.blog.content
            ))
        ]
        translation_content = self.llm.with_structured_output(Blog).invoke(messages)
        return {"blog": {"content": translation_content}}

    def route(self, state: BlogState) -> dict:
        """Prepare routing information."""
        return {"current_language": state.current_language}

    def route_decision(self, state: BlogState) -> str:
        """Determine translation route based on language."""
        language_map = {
            "hindi": "hindi",
            "french": "french",
            "urdu": "urdu",
            "pashto": "pashto",
            "german": "german",
            "arabic": "arabic",
            "russian": "russian"
        }
        return language_map.get(state.current_language.lower(), "english")


In [ ]:
# Graph Builder
class GraphBuilder:
    def __init__(self, llm):
        self.llm = llm
        self.graph = StateGraph(BlogState)
        self.blog_node = BlogNode(llm)

    def build_graph(self, with_translations: bool = False):
        """Build the graph with or without translation nodes."""
        # Common nodes
        self.graph.add_node("title_creation", self.blog_node.title_creation)
        self.graph.add_node("content_generation", self.blog_node.content_generation)
        
        # Base workflow
        self.graph.add_edge(START, "title_creation")
        self.graph.add_edge("title_creation", "content_generation")

        if with_translations:
            self._add_translation_nodes()
            self.graph.add_edge("content_generation", "route")
            self.graph.add_conditional_edges(
                "route",
                self.blog_node.route_decision,
                {lang: f"{lang}_translation" for lang in [
                    "hindi", "french", "urdu", "pashto", "german", "arabic", "russian"
                ]}
            )
            for lang in ["hindi", "french", "urdu", "pashto", "german", "arabic", "russian"]:
                self.graph.add_edge(f"{lang}_translation", END)
        else:
            self.graph.add_edge("content_generation", END)

        return self.graph.compile()

    def _add_translation_nodes(self):
        """Add translation nodes to the graph."""
        self.graph.add_node("route", self.blog_node.route)
        translations = [
            ("hindi", "hindi"),
            ("french", "french"),
            ("urdu", "urdu"),
            ("pashto", "pashto"),
            ("german", "german"),
            ("arabic", "arabic"),
            ("russian", "russian")
        ]
        
        for lang_key, lang_value in translations:
            self.graph.add_node(
                f"{lang_key}_translation",
                lambda state, lang=lang_value: self.blog_node.translation(
                    BlogState(
                        topic=state.topic,
                        blog=state.blog,
                        current_language=lang
                    )
                )
            )

In [ ]:


# FastAPI App
app = FastAPI()

@app.post("/blogs")
async def generate_blog(request: Request):
    data = await request.json()
    topic = data.get("topic")
    language = data.get("language", "english").lower()

    if not topic:
        raise HTTPException(status_code=400, detail="Topic is required")

    try:
        # Initialize components
        groq_llm = GroqLLM()
        graph_builder = GraphBuilder(groq_llm.get_llm())
        
        # Prepare initial state
        initial_state = BlogState(
            topic=topic,
            current_language=language,
            blog=Blog()
        )

        # Build and invoke graph
        graph = graph_builder.build_graph(with_translations=language != "english")
        result = graph.invoke(initial_state)
        
        return {"data": result}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    uvicorn.run("app:app", host="0.0.0.0", port=8000, reload=True)